In [1]:
%%capture
!pip install unsloth "xformers==0.0.28.post2" #"transformers==4.45.2"
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [2]:
# One must patch the DPO Trainer first!
from unsloth import PatchDPOTrainer
PatchDPOTrainer()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "ihughes15234/phi_3_5_mini_kp_12k_cfr_sft",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

#if error update tokenizer file on hf, error with unsloth writing this

==((====))==  Unsloth 2024.11.6: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.65G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/3.71k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/447 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 64,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.11.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep


In [5]:
!pip install huggingface_hub
!pip install wandb

In [6]:
from google.colab import userdata

from huggingface_hub import login

hf_token = userdata.get('HF_HUB_TOKEN') #   wanddb
login(hf_token)

In [7]:
# This code block is used to load a tokenizer from the Hugging Face Model Hub.

# 'tokenizer_id' is set to the 'model_id', which is the identifier for the pre-trained model.
# This assumes that the tokenizer associated with the model has the same identifier as the model.
from datasets import load_dataset
from transformers import AutoTokenizer

dataset = load_dataset("ihughes15234/kp_cfr_drpo_1200_v2", split = "train")

tokenizer_id = 'microsoft/Phi-3.5-mini-instruct'

# 'AutoTokenizer.from_pretrained' is a method that loads a tokenizer from the Hugging Face Model Hub.
# 'tokenizer_id' is passed as an argument to specify which tokenizer to load.
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)

# 'tokenizer.padding_side' is a property that specifies which side to pad when the input sequence is shorter than the maximum sequence length.
# Setting it to 'right' means that padding tokens will be added to the right (end) of the sequence.
# This is done to prevent warnings that can occur when the padding side is not explicitly set.
tokenizer.pad_token = tokenizer.unk_token  # use unk rather than eos token to prevent endless generation
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = 'right'

README.md:   0%|          | 0.00/468 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/45.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1195 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
# dataset_chatml = dataset.train_test_split(test_size=0.05, seed=1234)

In [8]:
import pprint
row = dataset[8]
pprint.pprint(row["prompt"])
pprint.pprint(row["chosen"])
pprint.pprint(row["rejected"])

('<|system|>\n'
 'You are a powerful gaming agent who can make proper decisions to beat the '
 'user in gaming tasks. You are a helpful assistant that strictly follows the '
 "user's instructions.<|end|>\n"
 '<|user|>\n'
 'Kuhn poker is a simple model zero-sum two-player imperfect-information game, '
 'amenable to a complete game-theoretic analysis. In Kuhn poker, the deck '
 'includes only three playing cards: a King (K), a Queen (Q), and a Jack (J).\n'
 'One card is dealt to each player, and the third is put aside unseen. The '
 'players take turns either <Bet> to match the bet raised by the opponent or '
 '<Pass> to concede the game.\n'
 'If a player bets, the other player must either call the bet by matching it '
 'or fold by conceding the game. If both players pass, the game is over, and '
 'the player with the higher-ranking card wins. The card rankings are as '
 'follows: King (K) > Queen (Q) > Jack (J).\n'
 '\n'
 'You are playing Kuhn poker with the opponent. The actions are de

In [9]:
# 'import wandb' is used to import the wandb library.
import wandb

# 'wandb.login()' is a method that logs you into your Weights & Biases account.
# If you're not already logged in, it will prompt you to log in.
# Once you're logged in, you can use Weights & Biases to track and visualize your experiments.
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

<a name="Train"></a>
### Train the model

In [10]:
# One must patch the DPO Trainer first!
from unsloth import PatchDPOTrainer
PatchDPOTrainer()

In [11]:
from transformers import TrainingArguments
from trl import DPOTrainer, DPOConfig
from unsloth import is_bfloat16_supported

dpo_trainer = DPOTrainer(
    model = model,
    ref_model = None,
    args = DPOConfig(
        per_device_train_batch_size = 16,
        per_device_eval_batch_size=8,
        gradient_accumulation_steps = 2,
        warmup_ratio = 0.1,
        num_train_epochs = 3,
        learning_rate = 5e-6,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.0,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
    ),
    beta = 0.1,
    train_dataset = dataset,
    tokenizer = tokenizer,
    max_length = 1024,
    max_prompt_length = 1024,
)



Extracting prompt from train dataset:   0%|          | 0/1195 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1195 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1195 [00:00<?, ? examples/s]

In [12]:
# This code block is used to initialize a Weights & Biases (wandb) run.

# 'project_name' is set to the name of the project in Weights & Biases.
project_name = "phi35-kp_dpo_3epoch_v2"

# 'wandb.init' is a method that initializes a new Weights & Biases run.
# 'project' is set to 'project_name', meaning that the run will be associated with this project.
# 'name' is set to "phi-3-mini-ft-py-3e", which is the name of the run.
# Each run has a unique name which can be used to identify it in the Weights & Biases dashboard.
wandb.init(project=project_name, name = "phi35-kp_dpo_3epocho_v2")

wandb: Currently logged in as: ihughes15234 (ihughes). Use `wandb login --relogin` to force relogin


In [13]:
dpo_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,195 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 2
\        /    Total batch size = 32 | Total steps = 111
 "-____-"     Number of trainable parameters = 119,537,664
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / rejected,logps / chosen,logits / rejected,logits / chosen
1,0.693100,0.000000,0.000000,0.000000,0.000000,-47.713150,-47.800201,4.011911,4.493264
2,0.693100,0.000000,0.000000,0.000000,0.000000,-47.311073,-47.780891,4.390520,4.748962
3,0.708500,-0.042220,-0.012827,0.312500,-0.029393,-48.619537,-48.658344,4.073396,3.917144
4,0.705000,0.009974,0.030133,0.281250,-0.020160,-47.639542,-47.302933,4.520902,4.544393
5,0.675000,0.026624,-0.013498,0.687500,0.040122,-47.893566,-48.105560,3.678766,4.029016
6,0.676800,0.040994,-0.001996,0.593750,0.042990,-48.080616,-47.119682,4.359056,4.557434
7,0.705600,-0.009586,0.009897,0.437500,-0.019483,-48.549484,-47.873184,3.903392,4.117945
8,0.683400,0.036522,0.010932,0.625000,0.025590,-48.046577,-47.261322,4.484426,4.518788
9,0.720500,0.013554,0.060137,0.312500,-0.046583,-47.516670,-47.729626,4.300853,4.201868
10,0.702700,0.013721,0.026359,0.531250,-0.012638,-48.251873,-47.745796,3.960184,4.096713


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / rejected,logps / chosen,logits / rejected,logits / chosen
1,0.693100,0.000000,0.000000,0.000000,0.000000,-47.713150,-47.800201,4.011911,4.493264
2,0.693100,0.000000,0.000000,0.000000,0.000000,-47.311073,-47.780891,4.390520,4.748962
3,0.708500,-0.042220,-0.012827,0.312500,-0.029393,-48.619537,-48.658344,4.073396,3.917144
4,0.705000,0.009974,0.030133,0.281250,-0.020160,-47.639542,-47.302933,4.520902,4.544393
5,0.675000,0.026624,-0.013498,0.687500,0.040122,-47.893566,-48.105560,3.678766,4.029016
6,0.676800,0.040994,-0.001996,0.593750,0.042990,-48.080616,-47.119682,4.359056,4.557434
7,0.705600,-0.009586,0.009897,0.437500,-0.019483,-48.549484,-47.873184,3.903392,4.117945
8,0.683400,0.036522,0.010932,0.625000,0.025590,-48.046577,-47.261322,4.484426,4.518788
9,0.720500,0.013554,0.060137,0.312500,-0.046583,-47.516670,-47.729626,4.300853,4.201868
10,0.702700,0.013721,0.026359,0.531250,-0.012638,-48.251873,-47.745796,3.960184,4.096713


TrainOutput(global_step=111, training_loss=0.5208901172285682, metrics={'train_runtime': 639.1218, 'train_samples_per_second': 5.609, 'train_steps_per_second': 0.174, 'total_flos': 0.0, 'train_loss': 0.5208901172285682, 'epoch': 2.96})

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

text = "Connect 4 is a two-player connection board game, where the players choose a color and then take turns dropping colored discs into a vertically suspended grid. The pieces fall straight down, occupying the next available space within the column. The objective of the game is to be the first to form a horizontal, vertical, or diagonal line of four of one's own discs. You are a gaming agent that aims to beat me in Connect 4 games. Each move is represented by a string consisting of two parts: the column (C) and the row (R), in that order. For instance, C1 means the first column. Your opponent has finished moves: <C6>,<C6>,<C1>,<C1>,<C3>,<C5>,<C4>,<C4> You have finished moves: <C4>,<C4>,<C6>,<C1>,<C7>,<C3>,<C3>,<C6> Currently, the legal positions are <C1>,<C2>,<C3>,<C4>,<C5>,<C6>,<C7> You must choose an legal action to set up advantages. \n \n Your output must be in the following format: \n \n Action: \n Your action wrapped with <>, <Cx>, e.g., <C1>, <C7> \n \n Please return your answer without explanation!"
text2 = "Breakthrough is a two-player game played on a rectangular board. Players take turns moving their pieces, which can move one space straight or diagonally forward if the target square is empty. A piece can also move diagonally forward to capture an opponent's piece. Capturing is optional, and a player can only capture one piece per turn. The goal is to be the first to reach the opponent's home row, the farthest row from the player. If all of a player's pieces are captured, they lose. The game does not allow draws, as pieces can only move forward or be captured.The Breakthrough board is identified by columns labeled start from A (from left to right) and rows numbered 1 to 8 (from bottom to top). The intersection of a column and a row specifies a unique square on the board.\n\nThe board now looks like :\n['8bbb', '7b..', '6...', '5...', '4bwb', '3.w.', '2w..', '1ww.'] \nAmong which, the letter 'b' represents black piece, while the letter 'w' represents white piece.\n And the character '.' represents vacant space.\n And the numbers in the board are the indexes of the rows.\nYour opponent has finished actions: <c2->b3> and <b3->a4> and <c1->c2> and <b2->a3> and <c2->b3> and <a3->b4>. You have finished actions: <b7->c6>, <c7->b6>, <b6->c5>, <c6->b5>, <b5->a4*>, <c5->c4>.\n\nCurrently, the legal actions are: <a8->b7> or <b8->b7> or <b8->c7> or <c8->b7> or <c8->c7> or <a7->a6> or <a7->b6> or <a4->a3> or <a4->b3*> or <c4->b3*> or <c4->c3>.\nYou must choose an legal action to set up advantages.\n\nYour output must be in the following format:\n\nAction:\nYour action wrapped with <>, <[a-c][1-8]->[a-c][1-8]>, e.g., <a7->a6>\n\nPlease return your answer without explanation!"
messages = [
    {"from": "human", "value": text2},# dataset_chatml['train'][63]["text"]
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


["<s> [INST] Breakthrough is a two-player game played on a rectangular board. Players take turns moving their pieces, which can move one space straight or diagonally forward if the target square is empty. A piece can also move diagonally forward to capture an opponent's piece. Capturing is optional, and a player can only capture one piece per turn. The goal is to be the first to reach the opponent's home row, the farthest row from the player. If all of a player's pieces are captured, they lose. The game does not allow draws, as pieces can only move forward or be captured.The Breakthrough board is identified by columns labeled start from A (from left to right) and rows numbered 1 to 8 (from bottom to top). The intersection of a column and a row specifies a unique square on the board.\n\nThe board now looks like :\n['8bbb', '7b..', '6...', '5...', '4bwb', '3.w.', '2w..', '1ww.'] \nAmong which, the letter 'b' represents black piece, while the letter 'w' represents white piece.\n And the c

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [14]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if True: model.push_to_hub_merged("phi35_kp_dpo3epoch_v2_1200", tokenizer, save_method = "merged_16bit")#, token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 7.6G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 47.86 out of 83.48 RAM for saving.


100%|██████████| 32/32 [00:00<00:00, 79.76it/s]


Unsloth: Saving to organization with address ihughes15234/phi35_kp_dpo3epoch_v2_1200
Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving to organization with address ihughes15234/phi35_kp_dpo3epoch_v2_1200
Unsloth: Uploading all files... Please wait...


  0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.65G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/None/phi35_kp_dpo3epoch_v2_1200


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)
10. [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
11. [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)
12. [**NEW**] We make Mistral NeMo 12B 2x faster and fit in under 12GB of VRAM! [Mistral NeMo notebook](https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>